In [39]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem
from sympy import Rational
import dill

dill.settings["recurse"] = True
from sympy import Rational as R

# Necessary symbols

In [59]:
x, y, x_1, y_1, x_2, y_2, x_3, y_3 = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")
p1, p2, p3 = sympy.symbols("p1:4")
right_vals = sympy.Matrix(sympy.symbols("f_1:10"))

In [41]:
q1 = p1**2 * (R(3) - R(2) * p1) + R(2) * p1 * p2 * p3
q1 = q1.subs(p3, R(1) - p1 - p2)

q2 = p2**2 * (R(3) - R(2) * p2) + R(2) * p1 * p2 * p3
q2 = q2.subs(p3, R(1) - p1 - p2)

q3 = p3**2 * (R(3) - R(2) * p3) + R(2) * p1 * p2 * p3
q3 = q3.subs(p3, R(1) - p1 - p2)

r1 = p1**2 * (p1 - R(1)) - p1 * p2 * p3
r1 = r1.subs(p3, R(1) - p1 - p2)

r2 = p2**2 * p1 + R(1, 2) * p1 * p2 * p3
r2 = r2.subs(p3, R(1) - p1 - p2)

r3 = p3**2 * p1 + R(1, 2) * p1 * p2 * p3
r3 = r3.subs(p3, R(1) - p1 - p2)

s1 = p1**2 * p2 + R(1, 2) * p1 * p2 * p3
s1 = s1.subs(p3, R(1) - p1 - p2)

s2 = p2**2 * (p2 - R(1)) - p1 * p2 * p3
s2 = s2.subs(p3, R(1) - p1 - p2)

s3 = p3**2 * p2 + R(1, 2) * p1 * p2 * p3
s3 = s3.subs(p3, R(1) - p1 - p2)

hermit_basis = [q3, r3, s3, q1, r1, s1, q2, r2, s2]

hermit_basis = sympy.Matrix([itm.subs(p1, x).subs(p2, y) for itm in hermit_basis])

In [42]:
delta = x_1 * y_2 - x_1 * y_3 - x_2 * y_1 + x_2 * y_3 + x_3 * y_1 - x_3 * y_2

P_1_x = (-y_1 + y_3) / delta
P_1_y = (+x_1 - x_3) / delta

P_2_x = (+y_1 - y_2) / delta
P_2_y = (-x_1 + x_2) / delta

J = sympy.Matrix(
    [
        [P_1_x, P_1_y],
        [P_2_x, P_2_y],
    ]
)

In [43]:
final_parameters = [x_1, y_1, x_2, y_2, x_3, y_3]

In [44]:
N = len(hermit_basis)

In [47]:
J11, J12, J21, J22 = sympy.symbols("J11, J12, J21, J22")
J_simp = sympy.Matrix([
    [J11, J12],
    [J21, J22]
])

result_A = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = hermit_basis[idx]
    second = hermit_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(x) * J_simp.row(0)[0] + first.diff(y) * J_simp.row(1)[0]
    u_y = first.diff(x) * J_simp.row(0)[1] + first.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------
    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(x) * J_simp.row(0)[0] + second.diff(y) * J_simp.row(1)[0]
    v_y = second.diff(x) * J_simp.row(0)[1] + second.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    A = u_x * v_x + u_y * v_y
    B = first * second

    sym_int_A = sympy.integrate(A, (y, 0, R(1) - x), (x, 0, R(1)))
    sym_int_B = sympy.integrate(B, (y, 0, R(1) - x), (x, 0, R(1)))

    result_A[idx][jdx] = sym_int_A
    result_B[idx][jdx] = sym_int_B

result_A = sympy.Matrix(result_A)
result_B = sympy.Matrix(result_B)

result_A = result_A * J.inv().det()
result_B = result_B * J.inv().det()

100%|██████████| 81/81 [01:20<00:00,  1.01it/s]


In [ ]:
result_A = sympy.Matrix(result_A)
result_B = sympy.Matrix(result_B)
right_vals = sympy.Matrix(right_vals)

In [48]:
result_A = result_A.subs({
        J11: J[0, 0],
        J12: J[0, 1],
        J21: J[1, 0],
        J22: J[1, 1]
})

result_B = result_B.subs({
        J11: J[0, 0],
        J12: J[0, 1],
        J21: J[1, 0],
        J22: J[1, 1]
})

In [50]:
M = sympy.diag(R(1), J.T, R(1), J.T, R(1), J.T).inv().T

In [53]:
FUNC_A = sympy.lambdify(final_parameters, M @ result_A @ M.T)

In [60]:
FUNC_B = sympy.lambdify([*final_parameters, *right_vals], M @ result_B @ M.T @ right_vals)

In [70]:
dill.dump(FUNC_A, open("../calculations/hermite_cubic_matrix", "wb"))
dill.dump(FUNC_B, open("../calculations/hermite_cubic_right", "wb"))